In [6]:
%run 10_kenlm_common.ipynb

/home/osx/anaconda3/envs/aac/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/osx/anaconda3/envs/aac/lib/python3.10/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [7]:
from datasets import load_dataset
#from g2p_en import G2p
from g2p import make_g2p
from num2words import num2words  # Import num2words from nltk
import re

# Load the Tatoeba dataset
tatoeba_dataset = load_dataset("tatoeba", 'en-mr')

# Initialize g2p converter
transducer = make_g2p('eng', 'eng-arpabet')

In [8]:
translations = tatoeba_dataset["train"]["translation"]

In [9]:
unique_phonemes = set()

with open("dataset_phonemes.txt", "w") as file:
    for translation in translations:
      sentence = translation["en"]
      sentence = remove_punctuation_and_special_characters(sentence)
      sentence = convert_numbers_to_words(sentence)
      phonemes_list = [transducer(word).output_string for word in re.findall(r'\S+', sentence)]
      all_phonemes = [item for sublist in phonemes_list for item in sublist.split()]

      # Update the set of unique phonemes excluding those containing hyphens
      unique_phonemes.update(phoneme for phoneme in all_phonemes if "-" not in phoneme)

      # Replace hyphens with letters that come after the hyphen
      all_phonemes_no_hyphen = [phoneme.split("-")[-1] if "-" in phoneme else phoneme for phoneme in all_phonemes]

      # Write all individual phonemes (without hyphens) to the file
      file.write(" ".join(all_phonemes_no_hyphen))
      file.write("\n")

# Print the vocabulary of phonemes
print("Phoneme Vocabulary:")
print(unique_phonemes)

Phoneme Vocabulary:
{'UW', 'NG', 'HH', 'AY', 'EY', 'W', 'M', 'F', 'R', 'N', 'T', 'G', ',,', ',', 'ER', 'Z', 'AW', 'SH', 'D', 'AA', 'Y', 'ZH', 'JH', 'EH', 'AO', 'L', 'K', 'TH', ',,,', 'AE', 'IY', 'OW', 'OY', 'S', 'B', 'DH', 'UH', 'V', 'P', 'CH', 'AH', 'IH'}


# Remove duplicates while preserving the order

In [10]:
# Read the content from the file
with open("dataset_phonemes.txt", 'r') as file:
    content = file.read()

# Remove instances of consecutive triple commas (,,,)
content = content.replace(',,,', '')

# Remove instances of consecutive double commas (,,)
content = content.replace(',,', '')

# Remove individual commas
content = content.replace(',', '')
# Write the modified content back to the file
with open('dataset_phonemes.txt', 'w') as file:
    file.write(content)
    
from collections import OrderedDict

# Read the content from the file
with open("dataset_phonemes.txt", 'r') as file:
    content = file.readlines()

# Remove duplicates while preserving the order
unique_content = list(OrderedDict.fromkeys(content))

# Write the modified content back to the file
with open('dataset_phonemes.txt', 'w') as file:
    file.writelines(unique_content)